In [1]:
import json
import pathlib

import tensorflow as tf

2021-11-30 19:57:54.047070: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-11-30 19:57:54.047112: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data_directory = './data_cache'
pathlib.Path(data_directory).mkdir(exist_ok=True)

data_file = 'recipes_raw.zip'
data_url = 'https://storage.googleapis.com/recipe-box/recipes_raw.zip'

tf.keras.utils.get_file(
    fname=data_file,
    origin=data_url,
    cache_dir=data_directory,
    extract=True,
    archive_format='zip'
)

53362688/53355492 [==============================] - 1s 0us/step

In [3]:
def load_data_from_file():
    file_names = [
        'recipes_raw_nosource_ar.json',
        'recipes_raw_nosource_epi.json',
        'recipes_raw_nosource_fn.json',
    ]

    data = []

    for file_name in file_names:
        file_path = f'{data_directory}/datasets/{file_name}'

        with open(file_path) as dataset_file:
            json_data = json.load(dataset_file)
            json_data_list = list(json_data.values())
            dict_keys = [key for key in json_data_list[0]]
            dict_keys.sort()
            data += json_data_list

    return data


data_raw = load_data_from_file()

./tmp/datasets/recipes_raw_nosource_ar.json
Number of examples:  39802 

Example object keys:
 ['ingredients', 'instructions', 'picture_link', 'title'] 

Example object:
 {'title': 'Slow Cooker Chicken and Dumplings', 'ingredients': ['4 skinless, boneless chicken breast halves ADVERTISEMENT', '2 tablespoons butter ADVERTISEMENT', '2 (10.75 ounce) cans condensed cream of chicken soup ADVERTISEMENT', '1 onion, finely diced ADVERTISEMENT', '2 (10 ounce) packages refrigerated biscuit dough, torn into pieces ADVERTISEMENT', 'ADVERTISEMENT'], 'instructions': 'Place the chicken, butter, soup, and onion in a slow cooker, and fill with enough water to cover.\nCover, and cook for 5 to 6 hours on High. About 30 minutes before serving, place the torn biscuit dough in the slow cooker. Cook until the dough is no longer raw in the center.\n', 'picture_link': '55lznCYBbs2mT8BTx6BTkLhynGHzM.S'} 

Required keys:

  title:  Slow Cooker Chicken and Dumplings 

  ingredients:  ['4 skinless, boneless chicke

In [16]:
def filter_incomplete_recipes(recipe):
    required_keys = ['title', 'ingredients', 'instructions']

    if not recipe:
        return False

    for required_key in required_keys:
        if not recipe[required_key]:
            return False

        if type(recipe[required_key]) == list and len(recipe[required_key]) == 0:
            return False

    return True


data = [recipe for recipe in data_raw if filter_incomplete_recipes(recipe)]

In [9]:
TITLE_STOP_WORD = '🆃🅸🆃🅻🅴\n'
INGREDIENTS_STOP_WORD = '🅸🅽🅶🆁🅴🅳🅸🅴🅽🆃🆂\n\n'
INSTRUCTIONS_STOP_WORD = '🅸🅽🆂🆃🆁🆄🅲🆃🅸🅾🅽🆂\n\n'


def recipe_to_string(recipe):
    # This string is presented as a part of recipes so we need to clean it up.
    noise_string = 'ADVERTISEMENT'

    title = recipe['title']
    ingredients = recipe['ingredients']
    instructions = recipe['instructions'].split('\n')

    ingredients_string = ''
    for ingredient in ingredients:
        ingredient = ingredient.replace(noise_string, '')
        if ingredient:
            ingredients_string += f'• {ingredient}\n'

    instructions_string = ''
    for instruction in instructions:
        instruction = instruction.replace(noise_string, '')
        if instruction:
            instructions_string += f'▪︎ {instruction}\n'

    return f'{TITLE_STOP_WORD}{title}\n{INGREDIENTS_STOP_WORD}{ingredients_string}{INSTRUCTIONS_STOP_WORD}{instructions_string}'


data = [recipe_to_string(recipe) for recipe in data]

In [12]:
MAX_RECIPE_LENGTH = 1500


def filter_recipes_by_length(recipe_test):
    return len(recipe_test) <= MAX_RECIPE_LENGTH


data = [recipe_text for recipe_text in data if filter_recipes_by_length(recipe_text)]

Dataset size BEFORE filtering:  122938
Dataset size AFTER filtering:  100212
Number of eliminated recipes:  22726


In [14]:
STOP_SIGN = '␣'

tokenizer = tf.keras.preprocessing.text.Tokenizer(
    char_level=True,
    filters='',
    lower=False,
    split=''
)

tokenizer.fit_on_texts([STOP_SIGN])
tokenizer.fit_on_texts(data)
tokenizer.get_config()

VOCABULARY_SIZE = len(tokenizer.word_counts) + 1

data_vectorized = tokenizer.texts_to_sequences(data)

{'num_words': None,
 'filters': '',
 'lower': False,
 'split': '',
 'char_level': True,
 'oov_token': None,
 'document_count': 100213,
 'word_counts': '{"\\u2423": 1, "\\ud83d\\udcd7": 100212, " ": 17527888, "S": 270259, "l": 3815150, "o": 5987496, "w": 964459, "C": 222831, "k": 890982, "e": 9296022, "r": 4760887, "h": 2922100, "i": 4911812, "c": 2883507, "n": 5304396, "a": 6067157, "d": 3099679, "D": 63999, "u": 2717050, "m": 1794411, "p": 2679164, "g": 1698670, "s": 4704222, "\\n": 1955281, "\\ud83e\\udd55": 100212, "\\u2022": 922813, "4": 232607, ",": 1130487, "b": 1394803, "t": 5997722, "v": 746785, "2": 493933, "(": 144985, "1": 853931, "0": 145119, ".": 1052548, "7": 31098, "5": 154071, ")": 144977, "f": 1042981, "y": 666553, "\\ud83d\\udcdd": 100212, "\\u25aa": 331058, "\\ufe0e": 331058, "P": 200597, "6": 51398, "H": 43936, "A": 134274, "3": 213519, "R": 101253, "x": 201286, "/": 345257, "I": 81591, "L": 46138, "8": 55352, "9": 17697, "B": 123813, "M": 78684, "F": 104359, "j": 1

{'num_words': None,
 'filters': '',
 'lower': False,
 'split': '',
 'char_level': True,
 'oov_token': None,
 'document_count': 100213,
 'word_counts': '{"\\u2423": 1, "\\ud83d\\udcd7": 100212, " ": 17527888, "S": 270259, "l": 3815150, "o": 5987496, "w": 964459, "C": 222831, "k": 890982, "e": 9296022, "r": 4760887, "h": 2922100, "i": 4911812, "c": 2883507, "n": 5304396, "a": 6067157, "d": 3099679, "D": 63999, "u": 2717050, "m": 1794411, "p": 2679164, "g": 1698670, "s": 4704222, "\\n": 1955281, "\\ud83e\\udd55": 100212, "\\u2022": 922813, "4": 232607, ",": 1130487, "b": 1394803, "t": 5997722, "v": 746785, "2": 493933, "(": 144985, "1": 853931, "0": 145119, ".": 1052548, "7": 31098, "5": 154071, ")": 144977, "f": 1042981, "y": 666553, "\\ud83d\\udcdd": 100212, "\\u25aa": 331058, "\\ufe0e": 331058, "P": 200597, "6": 51398, "H": 43936, "A": 134274, "3": 213519, "R": 101253, "x": 201286, "/": 345257, "I": 81591, "L": 46138, "8": 55352, "9": 17697, "B": 123813, "M": 78684, "F": 104359, "j": 1

In [ ]:
data_vectorized_padded_without_stops = tf.keras.preprocessing.sequence.pad_sequences(
    data_vectorized,
    padding='post',
    truncating='post',
    # We use -1 here and +1 in the next step to make sure
    # that all recipes will have at least 1 stops sign at the end,
    # since each sequence will be shifted and truncated afterwards
    # (to generate X and Y sequences).
    maxlen=MAX_RECIPE_LENGTH - 1,
    value=tokenizer.texts_to_sequences([STOP_SIGN])[0]
)

data_vectorized_padded = tf.keras.preprocessing.sequence.pad_sequences(
    data_vectorized_padded_without_stops,
    padding='post',
    truncating='post',
    maxlen=MAX_RECIPE_LENGTH + 1,
    value=tokenizer.texts_to_sequences([STOP_SIGN])[0]
)